In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import geojson
import numpy as np
import json

In [2]:
electricity_df = pd.read_csv('Electricity.csv', sep = ';')
electricity_df.head()

,Unnamed: 0,all renewables,Unnamed: 2,excluding hydropower,all renewables 2015,Unnamed: 5,% difference 2015 vs 2020,all electricity generation,all renewables.1,non-hydro renewables,story,story source
0,NaN,2020 or latest year,NaN,NaN,NaN,NaN,NaN,GWh,GWh,GWh,NaN,https://www.eia.gov/international/data/world/e...
1,Afghanistan,82.8%,82.8%,3.8%,86.1%,NaN,-3.29%,1.1,0.9,0.04,NaN,NaN
2,Albania,100.0%,100.0%,0.4%,100.0%,NaN,0.00%,5.2,5.2,0.02,NaN,NaN
3,Algeria,1.1%,1.1%,0.9%,0.3%,NaN,0.79%,76.7,0.8,0.69,NaN,NaN
4,American Samoa,0.0%,0.0%,0.0%,NaN,NaN,NaN,0.2,0.0,0.00,NaN,NaN


In [3]:
renewables_df = electricity_df[['Unnamed: 0', 'all renewables', 'excluding hydropower']]
renewables_df = renewables_df.drop(labels=0, axis=0)

renewables_df = renewables_df.rename(columns={'Unnamed: 0': 'country'})
renewables_df = renewables_df[renewables_df['country'].notna()]

renewables_df['all renewables'] = renewables_df['all renewables'].str.replace('%', '')
renewables_df['all renewables'] = renewables_df['all renewables'].astype(float)

renewables_df['excluding hydropower'] = renewables_df['excluding hydropower'].str.replace('%', '')
renewables_df['excluding hydropower'] = renewables_df['excluding hydropower'].astype(float)

# renewables_df.head()

In [4]:
renewables_df.iloc[[110]]

,country,all renewables,excluding hydropower
111,Libya,0.0,0.0


In [5]:
world = json.load(open('countries.geojson', 'r'))
# world['features'][1]

In [8]:
conv_dict = {'Antigua and Barbuda': 'Antigua & Barbuda',
    'Bosnia and Herzegovina': 'Bosnia & Herzegovina',
    'Ivory Coast': 'Côte d’Ivoire',
    'Democratic Republic of the Congo' : 'Congo Kinshasa',
    'Republic of Congo' : 'Congo Brazzaville',
    'Cape Verde' : 'Cabo Verde',
    'Guinea Bissau' : 'Guinea-Bissau',
    'Hong Kong S.A.R.' : 'Hong Kong',
    'Macedonia' : 'North Macedonia',
    'Macao S.A.R' : 'Macau',
    'Mauritania' : 'Western Sahara',
    'Republic of Serbia' : 'Serbia',
    'Sao Tome and Principe' : 'Sao Tome & Principe',
    'Swaziland' : 'Eswatini',
    'East Timor': 'Timor-Leste',
    'United Republic of Tanzania': 'Tanzania',
    'United States of America' : 'United States',
    'Saint Vincent and the Grenadines': 'Saint Vincent/Grenadines',
    'United States Virgin Islands' : 'U.S. Virgin Islands',
    'Chad' : 'Libya'
    
    }
# conv_dict = {'Anguilla', 'Aland', 'Andorra', 'Ashmore and Cartier Islands', 'French Southern and Antarctic Lands', 'Antigua and Barbuda', 'Bosnia and Herzegovina', 'Bajo Nuevo Bank (Petrel Is.)', 'Saint Barthelemy', 'Ivory Coast', 'Clipperton Island', 'Cyprus No Mans Area', 'Democratic Republic of the Congo', 'Republic of Congo', 'Cape Verde', 'Coral Sea Islands', 'CuraĂ§ao', 'Northern Cyprus', 'Dhekelia Sovereign Base Area', 'Federated States of Micronesia', 'Guernsey', 'Guinea Bissau', 'Hong Kong S.A.R.', 'Heard Island and McDonald Islands', 'Isle of Man', 'Indian Ocean Territories', 'British Indian Ocean Territory', 'Jersey', 'Baykonur Cosmodrome', 'Siachen Glacier', 'Kosovo', 'Liechtenstein', 'Luxembourg', 'Macao S.A.R', 'Saint Martin', 'Monaco', 'Marshall Islands', 'Macedonia', 'Northern Mariana Islands', 'Mauritania', 'Norfolk Island', 'Pitcairn Islands', 'Spratly Islands', 'Palau', 'Palestine', 'Western Sahara', 'Scarborough Reef', 'Serranilla Bank', 'South Georgia and South Sandwich Islands', 'San Marino', 'Somaliland', 'Republic of Serbia', 'Sao Tome and Principe', 'Swaziland', 'Sint Maarten', 'Chad', 'East Timor', 'Tuvalu', 'United Republic of Tanzania', 'United States Minor Outlying Islands', 'United States of America', 'US Naval Base Guantanamo Bay', 'Vatican', 'Saint Vincent and the Grenadines', 'United States Virgin Islands', 'Wallis and Futuna', 'Akrotiri Sovereign Base Area'}

In [9]:
# Instanciating necessary lists
found = []
missing = []
countries_geo = []

# For simpler acces, setting "zone" as index in a temporary dataFrame
tmp = renewables_df.set_index('country')

# Looping over the custom GeoJSON file
for country in world['features']:
    
    # Country name detection
    country_name = country['properties']['ADMIN']

    # Eventual replacement with our transition dictionnary
    country_name = conv_dict[country_name] if country_name in conv_dict.keys() else country_name
    go_on = country_name in tmp.index
    
    # If country is in original dataset or transition dictionnary
    if go_on:
        
        # Adding country to our "Matched/found" countries
        found.append(country_name)
        
        # Getting information from both GeoJSON file and dataFrame
        geometry = country['geometry']

        # Adding 'id' information for further match between map and data 
        countries_geo.append({
            'type': 'Feature',
            'geometry': geometry,
            'id':country_name
        })
        
    # Else, adding the country to the missing countries
    else:
        missing.append(country_name)

# Displaying metrics
print(f'Countries found    : {len(found)}')
print(f'Countries not found: {len(missing)}')
geo_world_ok = {'type': 'FeatureCollection', 'features': countries_geo}


Countries found    : 207
Countries not found: 48


In [10]:
spelled_countries = []
spelled_countries = renewables_df['country'].tolist()

# print(missing)
# print(spelled_countries)
# print(found)

In [11]:
# Create the log count column
renewables_df['renewables scale'] = np.log10(renewables_df['all renewables'])

# Get the maximum value to cap displayed values
max_log = renewables_df['renewables scale'].max()
max_val = int(max_log) + 1

renewables_df.head()

# Prepare the range of the colorbar
values = [i for i in range(max_val)]
ticks = [10**i for i in values]

c:\Users\anaku\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [12]:
# Create the log count column
renewables_df['renewables no hydropower scale'] = np.log10(renewables_df['excluding hydropower'])

# Get the maximum value to cap displayed values
max_log = renewables_df['renewables no hydropower scale'].max()
max_val = int(max_log) + 1

renewables_df.head()

# Prepare the range of the colorbar
values = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
ticks = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [13]:
renewables_df['renewables discrete'] = np.round(renewables_df['all renewables'], decimals = -1)

In [14]:
renewables_df['renewables discrete no hydropower'] = np.round(renewables_df['excluding hydropower'], decimals = -1)

In [16]:
import plotly.io as pio
pio.renderers.default = "browser"

In [17]:
colors = ['#de425b', '#ffffb1', '#488f31']

In [18]:
# Create figure
fig = px.choropleth_mapbox(
    renewables_df,
    geojson=geo_world_ok,
    locations='country',
    color=renewables_df['renewables discrete'],
    color_continuous_scale=colors,
    range_color=(0, 100),
    hover_name='country',
    hover_data={'renewables discrete': True, 'country': False, 'all renewables': True},
    mapbox_style='carto-positron',
    zoom=1.7,
    center={'lat': 25, 'lon': 15},
    opacity=0.75
)

# Define layout specificities
fig.update_layout(
    autosize=False,
    width=2000,
    height=1000,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4),
    coloraxis_colorbar={
        'title':'% of renewable energy including hydropower',
        'tickvals':values,
        'ticktext':ticks        
    }
)

# Display figure
fig.show()

In [19]:
renewables_df.iloc[[110]]

,country,all renewables,excluding hydropower,renewables scale,renewables no hydropower scale,renewables discrete,renewables discrete no hydropower
111,Libya,0.0,0.0,-inf,-inf,0.0,0.0


In [20]:
# Create figure
fig_no_hydropower = px.choropleth_mapbox(
    renewables_df,
    geojson=geo_world_ok,
    locations='country',
    color=renewables_df['renewables discrete no hydropower'],
    color_continuous_scale=colors,
    range_color=(0, 100),
    hover_name='country',
    hover_data={'renewables discrete no hydropower': False, 'country': False, 'excluding hydropower': True},
    mapbox_style='carto-positron',
    zoom=1.7,
    center={'lat': 25, 'lon': 15},
    opacity=0.75
)

# Define layout specificities
fig_no_hydropower.update_layout(
    autosize=False,
    width=2000,
    height=1000,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4),
    coloraxis_colorbar={
        'title':'% of renewable energy excluding hydropower',
        'tickvals':values,
        'ticktext':ticks        
    }
)

# Display figure
fig_no_hydropower.show()

In [21]:
import dash
from dash import dcc
from dash import html
from dash import Dash, dcc, html, Input, Output

In [22]:
app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])


app.layout = html.Div([
    html.H4('Renewable electricity by country'),
    html.P("Include or exclude hydropower:"),
    dcc.RadioItems(
        id='selection',
        options=["Renewables including hydropower", "Renewables excluding hydropower"],
        value='Renewables including hydropower',
    ),
    dcc.Loading(dcc.Graph(id="graph"), type="cube")
])


@app.callback(
    Output("graph", "figure"), 
    Input("selection", "value"))
    
def display_animated_graph(selection):
    renewables_df = px.data.gapminder() # replace with your own data source
    animations = {
        'Renewables including hydropower': fig,
        'Renewables excluding hydropower': fig_no_hydropower,
    }
    return animations[selection]


app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on
